## Time evolving foliar CN vs CO2, multiplicative model

These plots are similar to those in the CN_CO2Global notebook except that the equation relating foliar CN and CO2 is multiplicative instead of additive. 

### Read in data

In [1]:
#Load necessary packages for data visualization 
import xarray as xr
from ctsm_py import utils
#import utils
import numpy as np
import pandas as pd
from netCDF4 import num2date
import dask 

#some resources for plotting
import matplotlib
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os,sys
from matplotlib import cm
from matplotlib.colors import ListedColormap,LinearSegmentedColormap

%matplotlib inline
import pandas as pd
import datetime as dt
from datetime import datetime



In [2]:
# User defined options
model = 'CLM5_GSWP3'
## Read in slope 0 and slope 20 cases
archive = '/glade/scratch/wwieder/archive/'
caseShort = ['cnSlope0', 'cnSlope20', 'cn_flex_c']
cases = ['ctsm51d074_2deg_GSWP3V1_hist_'+short for short in caseShort]
path = '/lnd/hist/'
fin = [archive + case + path + "concat/" + case + '_cat.nc' for case in cases]
print(fin[0])

# Read in files for .area and .landfrac
fin2 = [archive + case + path + case + '.clm2.h0.1850-01.nc' for case in cases]
print(fin2[0])

## read in the CNMulti case
months = pd.date_range(start='1850-1', end= '2015-1', freq = 'M').strftime('%Y-%m')
case = 'ctsm51d074_2deg_GSWP3V1_hist_cnMulti'
fin3 = [archive + case + path + case + '.clm2.h0.' + months + '.nc']

/glade/scratch/wwieder/archive/ctsm51d074_2deg_GSWP3V1_hist_cnSlope0/lnd/hist/concat/ctsm51d074_2deg_GSWP3V1_hist_cnSlope0_cat.nc
/glade/scratch/wwieder/archive/ctsm51d074_2deg_GSWP3V1_hist_cnSlope0/lnd/hist/ctsm51d074_2deg_GSWP3V1_hist_cnSlope0.clm2.h0.1850-01.nc


In [3]:
# Read in data, time and time_bounds just used for utils.time_set_mid
var = ['NPP', 'GPP', 'LEAFCN', 'EFLX_LH_TOT', 'TOTVEGC', 'ELAI', 'TOTLITC','TOTLITN','FPI', 'FFIX_TO_SMINN','NBP', 'NET_NMIN','NFIX',
       'time', 'time_bounds']

'''select the variables we want to read in'''
def preprocess(ds, var=var):
    return ds[var]

# For slope 0
for i in range(len(cases)):
    dsIN = utils.time_set_mid(xr.open_mfdataset(fin[i], decode_times=True, preprocess = preprocess), 'time')
    LandDs = utils.time_set_mid(xr.open_mfdataset(fin2[i], decode_times=True), 'time')
    dsIN['area'] = LandDs.area
    dsIN['landfrac'] = LandDs.landfrac

    if i == 0:
        ds = dsIN
    else:
        ds = xr.concat([ds,dsIN],"case",data_vars='all')


ds = ds.assign_coords(case=("case", caseShort))
ds = ds[['NPP', 'GPP', 'LEAFCN', 'FFIX_TO_SMINN','NBP', 'EFLX_LH_TOT', 'TOTVEGC', 'ELAI', 'TOTLITC','TOTLITN','NET_NMIN','FPI', 'NFIX']]
#ds = ds.load()  # this may make things really slow now, but hopefully faster later on
ds

<xarray.Dataset>
Dimensions:        (case: 3, time: 1980, lat: 96, lon: 144)
Coordinates:
  * time           (time) object 1850-01-16 11:44:59.999993 ... 2014-12-16 12...
  * lon            (lon) float32 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * lat            (lat) float32 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * case           (case) <U9 'cnSlope0' 'cnSlope20' 'cn_flex_c'
Data variables: (12/13)
    NPP            (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    GPP            (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    LEAFCN         (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    FFIX_TO_SMINN  (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    NBP            (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    EFLX_LH_TOT    (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    ...             ...
    ELAI           (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    TOTLITC        (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    TOTLITN        (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    NET_NMIN       (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    FPI            (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
    NFIX           (case, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 96, 144), meta=np.ndarray>
Attributes: (12/100)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              Mon Apr 25 05:33:00 2022: ncrcat ct...
    source:                               Community Terrestrial Systems Model
    hostname:                             cheyenne
    ...                                   ...
    cft_tropical_corn:                    61
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     month_1
    NCO:                                  netCDF Operators version 5.0.3 (Hom...

In [14]:
## Read in the CNMulti case data
var = ['NPP', 'GPP', 'LEAFCN', 'EFLX_LH_TOT', 'TOTVEGC', 'ELAI', 'TOTLITC','TOTLITN','FPI', 'FFIX_TO_SMINN','NBP', 'NET_NMIN','NFIX',
       'time', 'time_bounds']

'''select the variables we want to read in'''
def preprocess(ds2, var=var):
    return ds2[var]

#ds2 = utils.time_set_mid(xr.open_mfdataset(fin3, decode_times=True), 'time', engine = netCDF4)
ds2 = utils.time_set_mid(xr.open_mfdataset(fin3[0], combine='by_coords', decode_times=True, preprocess = preprocess), 'time')
#ds = ds.assign_coords(case=("case", caseShort))
#ds = ds[['NPP', 'GPP', 'LEAFCN', 'FFIX_TO_SMINN','NBP', 'EFLX_LH_TOT', 'TOTVEGC', 'ELAI', 'TOTLITC','TOTLITN','NET_NMIN','FPI', 'NFIX']]
#ds = ds.load()  # this may make things really slow now, but hopefully faster later on
#ds

In [15]:
ds2

<xarray.Dataset>
Dimensions:        (time: 1980, lat: 96, lon: 144, hist_interval: 2)
Coordinates:
  * time           (time) object 1850-01-16 11:44:59.999993 ... 2014-12-16 12...
  * lon            (lon) float32 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * lat            (lat) float32 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
Dimensions without coordinates: hist_interval
Data variables: (12/14)
    NPP            (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    GPP            (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    LEAFCN         (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    EFLX_LH_TOT    (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    TOTVEGC        (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    ELAI           (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    ...             ...
    FPI            (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    FFIX_TO_SMINN  (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    NBP            (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    NET_NMIN       (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    NFIX           (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    time_bounds    (time, hist_interval) object dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/99)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 06/08/22 09:24:42
    source:                               Community Terrestrial Systems Model
    hostname:                             cheyenne
    ...                                   ...
    cft_irrigated_switchgrass:            60
    cft_tropical_corn:                    61
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     month_1